# Math  1376: Programming for Data Science
---

In [ ]:
import numpy as np  # We use numpy in this lecture
import matplotlib.pyplot as plt
%matplotlib widget

## Module 04: Some useful applications of Modules 01-03
---

In this module, we will now pull together material across our first three modules to solve some practical problems. 

You may find it useful to review the notebooks in those modules beforehand or just simply open some of the notebooks to have their contents available to review as necessary. 

While there are a seemingly endless number of practical problems we can attempt to solve with what we have learned so far, we will focus on three ubiquitous problems in the computational sciences.

- Root-finding. (The content of the first part of the part-a lecture of this module.)

- Numerical integration. (The content of the second part of the part-a lecture of this module.)

- Optimization. (This content is pursued in the homework of this module.)

### A note about calculus concepts and interactive visualizations (i.e., widgets)
---

- These topics are commonly studied as applications of calculus concepts. 

- However, while we may make passing reference to certain calculus concepts, you do *not* need to know calculus to follow the narratives. (This, of course, is not to say that you should not seek to master calculus at some point.)

- Our focus is on the *big picture ideas* and we use interactive graphics (powered by a widgets module) to help us explore these ideas.

## Learning Objectives for Part (a)

- Understand what a root-finding problem means and how they arise in practice.


- Apply different types of root-finding algorithms to various functions written as `lambda` Python functions.


- Create code that implements root-finding algorithms of different types.


- Create annotations and interactive widgets to enhance visualizations of data.


- Understand what integration means and how it arises in practical applications.


- Implement different types of numerical integration algorithms to various functions written either as `lambda` Python functions or more complicated user-defined functions.


- Create code that implements numerical integration algorithms of different types.

### Some larger "in situ" learning objectives (i.e., learning that will occur by design of activities)

While we are going to explore how to implement some of these algorithms as activities below, our learning objectives go beyond simple correct implementation. We will also consider what it means to do the following:

- *compare* and *analyze* different algorithms developed for solving the same generic problem;

- use this comparison to *choose* the "right" algorithm for solving a *specific* problem;

- create a module (in the external activity) that encodes various algorithms and a *wrapper* function that automatically chooses which algorithm to apply based on the inputs.

Some of this is done in the notebook while other parts are left for homework.

## Notebook contents <a name='Contents'></a>
---

- [Part (a)(i): Root-finding](#Root-finding)

    - [Bracketing methods](#Bracketing)

    - [Activity 1: Implementing the bisection algorithm](#activity-bisection)

    - [Activity 2: Summary for root-finding](#activity-summary-roots)
    
- [Part (a)(ii): Numerical integration](#integration)

    - [Geometric (deterministic) methods](#rectangle-rules)

    - [Activity 3: Rectangle rules](#activity-rectangle-rules)
    
    - [Activity 4: Summary for numerical integration](#activity-summary-integration)

## Part (a)(i): Root-finding <a name='Root-finding'></a>
---

**Expected time to completion: 9 hours**
    

<mark> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('TBD', width=800, height=450)

### What does root-finding mean?
---

Simply put, given $f(x)$, a root of the function is some point $x=c$ such that $f(c)=0$. 

In other words, the ***roots*** of a function are the *inputs* that make the *output* equal to zero. 

### lambda functions for simple examples
---

We mentioned these briefly in 03_lecture_part_b notebook primarily in the context of list comprehensions, so you may want to review that briefly for some more context.

*A lambda function is very useful in the context of this notebook, so we dive into them a bit more here.*

What is a `lambda` function? You can read quite a bit about such functions [here](https://www.w3schools.com/python/python_lambda.asp) or [here](https://realpython.com/python-lambda/).

<mark> ***Key Points:*** </mark>

- The main takeaway is that if you have a simple type of "anonymous" function that you just need for a short period of time in your code, then a lambda function is probably right for you. It is *not* necessary to use these, but they are useful. 

- A generic use syntax we care about would look something like this:

  > ```f = lambda arguments : expression ```

In [ ]:
# Example of using a lambda function to create a multivariate linear function
f = lambda x, y, z : 3*x - 2*y + 4*z

In [ ]:
f(1, 2, 3)

**Run the code cells below to see an example of a `lambda` function in the context of plotting the roots of a polynomial.**

- Pay attention to how we use these types of functions in this lecture. At times, we treat these as completely anonymous functions in the sense that we pass them directly into algorithms without formally defining them anywhere else in the code. This allows us to study how the algorithms perform on different types of functions rapidly without having to formally declare/create the functions elsewhere in the code.

- In other words, they are simply a matter of convenience.

In [ ]:
f = lambda x : x**3 - x - 2  # defining f(x) = x^3-x-2

In [ ]:
# A crash-course in using plots to tell a story in two easy steps
%matplotlib widget

#####################################
# Step 1: Plot your stuff
#####################################
plt.figure(figsize=(10,5), num=0)

x = np.linspace(-1, 2, 100)  # Create 100 uniformly spaced points between -1 and 2
plt.plot(x, f(x), 'b', linewidth=2)  # plot of function f vs x

# Let's add some axes to the plot
plt.axvline(0, linewidth=1, linestyle=':', c='k')  # plot typical y-axis
plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

# Let's add the root to the plot using a scatter plot
c = 1.521  # approximate root
plt.scatter(c, f(c), s=70, c='r', marker='s')  # plot approximate root

######################################
# Step 2: Annotate and add informational text to explain your stuff
######################################
# Create a text string for the title that formats certain floating point numbers
# that are relevant in our plot
title_str = 'Approx. root at c=' + '{:.3}'.format(c) + r' with f(c)$\approx$' + '{:.2e}'.format(f(c))
plt.title(title_str, fontsize=14)

# Now we add fancier annotations to the plot with text and arrows in a two-step process

# First, we define the dict (dictionary) of properties of the text and arrows used in the annotation
bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.5, linewidth=2)  # properties for bounding box of text
arrowprops = dict(arrowstyle='->', color='k', alpha=0.5, linewidth=2)  # properties for the arrow

# Now annotate the plot
plt.annotate('Approx. root', fontsize=12, xy=(c,f(c)), xytext=(c-1.25,1), color='w',
             bbox = bbox, arrowprops = arrowprops)

plt.show();

### A plotting tool class for roots
---

For lots of different functions $f$, we may wish to repeat the above plotting routine, but we certainly don't want to write out all of this code again. 

Also, the above code had some "hard-coded" elements to it that were based on our prior knowledge about where a root of $x^3-x-2$ is located. 

In more general cases, we may want a plotting tool hat allows us to quickly explore where a potential root may be for a given function.

We *could* just "functionalize" the code above. However, we are interested in pursuing root-finding problems in this notebook where a particular function does not exist in isolation from specific types of data like its roots. When various "types of things" are intended to be used in an integrated manner, this is a good indication that we should at least consider the use of classes to organize these various "things" as data or method attributes of an object.

Thus, we create a class of plotting tools that is instantiated for a given function and useful for this lecture notebook, which also gives us some more experience working with classes.

In the code below, note how we have created a class with fairly short/modularized method attributes that each serve a specific purpose in relationship to the current state of knowledge we have regarding the root of a function we are interested in studying.

<mark> Suggested activity:</mark> 

- Add more information to the docstrings.

- Provide more useful code comments wherever they are missing.

In [ ]:
class PlotToolsForRoots(object):
    '''
    A class that provides useful plots/information involving the root of a function.
    '''
    def __init__(self, f, root=None):
        '''
        If a root is given, then it should be given as a float.
        '''
        self.f = f
        self.root = root  # May be None type if no root is known at time of initialization

        return
    
    def set_root(self, root):
        '''
        Used to set the self.root attribute as a new root value. Give as a float.
        '''
        self.root = root
        
        return
        
    def plot(self, x_min, x_max, fignum=0, N=100, c='b', ls='-', lw=2, show_root = True, annotate=True, clf = True):
        '''
        Used to visualize the function and potentially its root.
        '''
        self.fig = plt.figure(fignum)
        if clf is True:
            self.fig.clf()
        
        # Let's add x-axis to the plot 
        # We are not adding the y-axis b/c the root may be far away from x=0 which is where typical y-axis is plotted
        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
        
        x_plot = np.linspace(x_min, x_max, N)
        
        plt.plot(x_plot, self.f(x_plot), c=c, ls=ls, lw=lw)
        
        if self.root is not None and show_root:
            self.plot_root(annotate)
        
        plt.show();
        return
        
    def plot_root(self, annotate=True):
        '''
        Used to plot the root if it has been found/estimated
        '''
        
        plt.scatter(self.root, self.f(self.root), s=70, c='r', marker='s')  # plot approximate root

        # Create a text string for the title that formats certain floating point numbers
        # that are relevant in our plot
        title_str = 'Approx. root at c=' + '{:.3}'.format(self.root) + r' with f(c)$\approx$' + '{:.2e}'.format(self.f(self.root))
        plt.title(title_str, fontsize=14)
        
        if annotate:
            self.plot_annotate()
            
        return
    
    def plot_annotate(self):    
        '''
        Used to annotate the root plot with text and arrows in a two-step process
        '''

        # First, we define the dict (dictionary) of properties of the text and arrows used in the annotation
        bbox = dict(boxstyle="round", fc='b', color='r', alpha=0.5, linewidth=2)  # properties for bounding box of text
        arrowprops = dict(arrowstyle='->', color='k', alpha=0.5, linewidth=2)  # properties for the arrow

        # Now annotate the plot
        y_min, y_max = plt.ylim() #get min and max y-values from plot to offset annotations
        y_range = y_max - y_min
        x_min, x_max = plt.xlim()
        x_range = x_max - x_min

        plt.annotate('Approx. root', fontsize=12, xy=(self.root,self.f(self.root)), 
                     xytext=(self.root-0.35*x_range,0.1*y_range), 
                     color='w', bbox = bbox, arrowprops = arrowprops);
            
        return

In [ ]:
fplot = PlotToolsForRoots(f)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=1)

In [ ]:
fplot.set_root(1.521)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2, annotate=False)

In [ ]:
%matplotlib widget

fplot.plot(-1, 2, fignum=2, show_root=False)

In [ ]:
# Now we see an example that really illustrates why we do not want to plot the typical y-axis

fplot_2 = PlotToolsForRoots(lambda x : (x-2000)*(x-5000)*(x-10000))

In [ ]:
# Pay attention to the y-axis range, it is in units of 1e11 = 10,000,000,000.
# The "largest" value of the function here is about -10,000,000,000.
# In other words, for x between -10 and 10, the function never gets closer 
# than a distance of about 10 billion units from the x-axis.
# We are pretty far off from a root then.

%matplotlib widget
fplot_2.plot(-10, 10, fignum=3)  

In [ ]:
# A plot of x between 1000 and 110000 shows where all three roots are located and
# we also get the sense that the function is just a typical cubic polynomial.
# We do not require the typical y-axis here (which is plotted at x=0). We really
# only need to use the typical x-axis (i.e., y=0) in our plots to see where the function may have roots.

%matplotlib widget
fplot_2.plot(1000, 11000, fignum=4) 

In [ ]:
# Here, we do have knowledge of the exact location of a root at 2000

fplot_2.set_root(2000.)  # Try using int 2000 instead of the float 2000., can you explain the error?

%matplotlib widget
fplot_2.plot(1000, 3000, fignum=5)

***The above examples provide us more of the idea of the "what" but not the "why."***

### Why should we care about roots?
---

Is there anything particularly important about a function being zero? In general, the answer is no. However, it depends on the function and the goals of the problem/application for which the function models system behavior. 
The assignment discusses how root problems arise naturally in many optimization problems.
Below, we provide some simpler motivating examples.

<mark> ***Some motivating examples:*** </mark>

- Suppose there are two launched projectiles with two height functions denoted by $h_1(t)$ and $h_2(t)$, and we are very interested when, or even *if*, there is a time where $h_1(t)=h_2(t)$. In other words, is there a time where the objects collide? This is an important question that arises in missile defense systems, satellite monitoring, and autonomous vehicle guidance systems. If we define $f(t)=h_1(t)-h_2(t)$, then we are interested in knowing whether or not $f(t)=0$. 

- Suppose $p(t)$ describes a model you develop for the price of a potential investment (it could be in a savings/money market account, stock, or mutual fund to name just a few). You are probably interested in knowing when your model predicts the price to reach a certain target value, $p_{target}$, so that you can develop certain financial plans/goals around this time frame (e.g., having enough for a down payment on a car or house, take a vacation, or retire). Defining $f(t)=p(t)-p_{target}$ means you are interested in determining when $f(t)=0$.

### And these root problems are solved how?
---

There are lots of algorithms that attempt to approximate the roots of a function, e.g., see https://en.wikipedia.org/wiki/Root-finding_algorithms. 

<mark> ***Key Points:*** </mark>

- Most algorithms are *iterative*. (Meaning we need to write *loops* to implement them.)

- Some algorithms are very easy to implement. Some are more difficult. 
    
  Generally, the easier something is to implement, the more restrictive the conditions are under which we expect it to produce anything meaningful or the convergence rate sucks.

## Bracketing methods <a name='Bracketing'></a>
---

<mark> Run the code cell below and click the "play" button to see the second recorded lecture associated with this notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('TBD', width=800, height=450)

## Bisection algorithm (https://en.wikipedia.org/wiki/Bisection_method)
---

The basic idea is best illustrated by an interactive demo and playing the video above.

***Recall that we use widgets to create the interactivity. Widgets are very useful for illustrating concepts and exploring ideas.***

In [ ]:
# Usually place these at the top of a notebook if you know you are going to use widgets
# This cell only needs to be executed once
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

<mark> Suggested activity:</mark> If you really want to understand things, do the following:

- Update the docstring in `compute_bisection` to describe the (1) role of the various parameters, and (2) what variables (if any) the function returns.

- Add at least one doctest.

In [ ]:
def compute_bisection(f, a, b, n=10, tol_interval=1e-5, tol_f=1e-8):
    '''
    This simple function applies up to n iterations of the 
    bisection algorithm
    '''
    
    # Let us first check the conditions to apply the bisection
    # algorithm are satisfied
    if a >= b:
        raise ValueError('The bisec. alg. requires a<b')
    if f(a)*f(b) > 0:
        raise ValueError('The bisec. alg. requires f(a) and f(b) ' +\
                         'are different signs')
        
    # Now, perform the bisection algorithm
    current_iter = 0
    while current_iter < n:
        current_iter += 1
        
        # bisect the interval [a,b], i.e., compute mid-point
        c = (b+a)/2 
        
        # Check if c is an (approx.) root
        if np.abs(f(c)) < tol_f:
            break
        
        # Determine if a or b should be replaced with c
        if f(a)*f(c) < 0:
            b = c
        else:
            a = c
            
        # Check if dividing [a,b] in half made the interval 
        # too small to continue
        if (b-a) < tol_interval:
            break
        
    return (a, b, c)  # Returns a tuple

In [ ]:
# This uses the lambda function f previously defined
# as well as default values for three of the parameters
a, b, c = compute_bisection(f, 0, 2)

# Print the approximate root
print(c)

In [ ]:
# We can also return the tuple of outputs as a single variable
bisec_results = compute_bisection(f, 0, 2)

# The output is a tuple
print(type(bisec_results))

# The length of the tuple
print(len(bisec_results))

# The last component of the tuple has the approx. root
print(bisec_results[-1]) 

In [ ]:
# We could just ignore all the c value returned by the function as follows
_, _, c = compute_bisection(f, 0, 2)

print(c)

In [ ]:
# OR, we could just return only the last value of the tuple
c = compute_bisection(f, 0, 2)[2]

print(c)

In [ ]:
# If we did not know how many components were in the tuple, 
# BUT we did know that the last one had the value we were after, 
# then we could use the following line of code
c = compute_bisection(f, 0, 2)[-1]

print(c)

**Now let's add some interactivity.** Watch what if you choose `b` (or `a`) such that `f(b)` and `f(a)` have the same sign. But, how do we know when that will happen? (*Hint*: refer to the plot.)

In [ ]:
interact(compute_bisection, 
         f = fixed(f),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

**We can easily play with other lambda functions.**

***But,*** what is going on with all of the errors as we try different values of `b` in the code below for the lambda function used?

In [ ]:
interact(compute_bisection, 
         f = fixed(lambda x: np.sin(x**3 - x - 2)),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

**Let's plot the lambda function from the previous code cell to see what is going on.**

<mark> Suggested activity:</mark>

- After looking at the plots below (or after making your own), enter in more reasonable values for `a` and `b` in the widget above to see the different roots we could approximate *as long as we use suitable initial values for `a` and `b`*.

In [ ]:
fplot_new = PlotToolsForRoots(lambda x: np.sin(x**3 - x - 2))

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=0, x_max=2)  # Observe that f(x_min) is same sign as f(x_max)

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=1.25, x_max=1.75)  # We could try to find the root that is between 1.25 and 1.75

In [ ]:
%matplotlib widget
fplot_new.plot(x_min=1.75, x_max=2)  # We could also try to find the root that is between 1.75 and 2

**Let's plot a different function that never crosses the x-axis.**

In [ ]:
f_new = lambda x: x**4 - 2*x**3 + x + 1

fplot_new = PlotToolsForRoots(f_new)

In [ ]:
%matplotlib widget

fplot_new.plot(x_min=-2, x_max=3)

**We HOPE to get (in fact we *better get*) an error when we try to find a root to the function above.**

<mark> ***Key Points:*** </mark>

- The `compute_bisection` **is working PROPERLY if it gives us an error in this case.**

- In general, an error produced by a code does not mean that there is an error *in* the code. Many times, the code is operating *as expected* or at least *as intended*. 
    
  - Many errors are *user errors* where the user tries to use the code in a way it is not intended to be used, which causes an *unexpected error*. 

  - When an error is returned by the code that it is written to catch, such as in the `compute_bisection` algorithm, then this is an *expected error* that the programmer knows (either through domain-specific knowledge or by testing code with users and gathering feedback) may happen. 

In [ ]:
interact(compute_bisection, 
         f = fixed(f_new),
         a = widgets.FloatSlider(value=0, min=-1, max=2, step=0.1), 
         b = widgets.FloatSlider(value=2, min=0, max=3, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=100, step=1),
         tol_interval = fixed(1e-5),
         tol_f = fixed(1e-8))

***What is the bigger lesson here?*** 

Plots are *really* useful and we should often do some initial "grunt" work on examining the problem we are asked to solve to make sure that we actually set it up correctly. 

The `compute_bisection` was *not* failing just because it output errors. It was functioning just as it should. We gave it *bad* choices of `a` and `b` as the default values in the widget. Simply looking at the plot reveals this to us right away. 

While we may not always be able to plot a function for many reasons (e.g., it takes too long to evaluate the hundreds to thousands of times necessary to make a reasonable looking plot, the dimension of the input space is simply too high to properly visualize, etc.), **we should always try to visualize the data** associated with a problem *somehow* and in *someway* to orient ourselves properly on the problem.

### A `Bisection` class
---

<mark> Run the code cell below and click the "play" button to see the third recorded lecture associated with this notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('TBD', width=800, height=450)

With the above lesson in mind along with the broader perspective on the class of problems we are trying to solve, we now create a new class `Bisection` that starts to put all the useful attributes of a root-finding problem together based on the bisection algorithm. 

What should the attributes be?

- **Data attributes:**

  - The function `f`, the `a` and `b` parameters defining the interval $[a,b]$ where we look for a root, and `n` the number of iterations of the algorithm we will make are all obvious data attributes for this class. Since we may not know the `a`, `b`, and `n` we want to use for a particular `f`, we should default these to `None` upon initialization and then have a method attribute that can set these values similar to what we did for the `root` parameter with the method `set_root` in the `PlotToolsForRoots` class above.

  - One of the data attributes should be an instantiation of the `PlotToolsForRoots`. This data attribute, which is made from a class, is quite useful for us in determining the `a` and `b` parameters (and even the `n` parameter).
  
  - The root computed by the bisection algorithm is another data attribute, but it is not determined at initialization. It is determined and set when the bisection algorithm executes.
  
  - The tolerance parameters for the bisection algorithm are other data attributes, and these seem like the types of attributes that should have some default values based on "typical" use cases from prior experience (i.e., we default to values that "feel" right).

- **Method attributes:**

  - The compute bisection algorithm.
  
  - A method for setting the `a`, `b`, and `n` parameters to other values we determine are "better" than what was assigned (if anything) upon initialization. Such values may be determined by examining a plot using the data attribute associated with the `PlotToolsForRoots` class.

<mark> Suggested activity:</mark> 

- Add docstrings and code comments below.

In [ ]:
class Bisection(object):
    
    def __init__(self, f, a=None, b=None, n=None, tol_interval=1e-5, tol_f=1e-8):
        
        self.f = f
        self.a = a
        self.b = b
        self.n = n
        self.tol_interval = tol_interval
        self.tol_f = tol_f
        self.plot_tool = PlotToolsForRoots(f)
        
        return
        
    def set_bisection_parameters(self, a=None, b=None, n=None, tol_interval=None, tol_f=None):
        
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b
        if n is not None:
            self.n = n
        if tol_interval is not None:
            self.tol_interval = tol_interval
        if tol_f is not None:
            self.tol_f = tol_f
            
        return
            
    def compute_bisection(self):
        '''
        This simple function applies up to n iterations of the 
        bisection algorithm. 
        
        It sets new data attributes based on the outputs of the previously 
        defined compute_bisection standalone function.
        '''
        
        if (self.a is None) or (self.b is None) or (self.n is None):
            raise AttributeError('Check that all bisection parameters are set. \n' +\
                   'Use set_bisection_parameters to set: a, b, and/or n.')

        # Let us first check the conditions to apply the bisection
        # algorithm are satisfied
        if self.a >= self.b:
            raise ValueError('The bisec. alg. requires a<b')
        if self.f(a)*self.f(b) > 0:
            raise ValueError('The bisec. alg. requires f(a) and f(b)' +\
                             'are different signs')

        a_n = self.a
        b_n = self.b
        # Now, perform the bisection algorithm
        current_iter = 0
        while current_iter < self.n:
            current_iter += 1

            # bisect the interval [a,b], i.e., compute mid-point
            c = (b_n + a_n)/2 

            # Check if c is an (approx.) root
            if np.abs(self.f(c)) < self.tol_f:
                break

            # Determine if a or b should be replaced with c
            if self.f(a_n)*self.f(c) < 0:
                b_n = c
            else:
                a_n = c

            # Check if dividing [a,b] in half made the interval 
            # too small to continue
            if (b_n - a_n) < self.tol_interval:
                break
                
        # New data attributes are set based on the completion of the above algorithm        
        self.a_n = a_n 
        self.b_n = b_n
        self.root = c
        self.plot_tool.set_root(c)  # Useful for plotting!

        return

Let's now explore how easy it is to use this class to study a function and approximate its root.

In [ ]:
f_new = lambda x: x**3-x-2

f_bisect = Bisection(f_new)

In [ ]:
%matplotlib widget

f_bisect.plot_tool.plot(-1, 3)

In [ ]:
f_bisect.compute_bisection()  # <---- this will raise an attribute error, why?

In [ ]:
f_bisect.set_bisection_parameters(a = -1, b = 3, n = 10)

In [ ]:
f_bisect.compute_bisection()

In [ ]:
%matplotlib widget

f_bisect.plot_tool.plot(-1, 3)

### Let's explore how to visualize the entire bisection algorithm

Below, we create a plotting function that *wraps* around the `compute_bisection` algorithm to add some illustration to the actual bisection process. 

**Read the comments in the code cell below.**

Note: We first present this as a standalone function for clarity before we include it in a class.

In [ ]:
# This cell creates a plotting function that we 
# then use widgets to interact with

def plot_bisection(f, a, b, n, x_min, x_max, grid_pts = 100, show_final_interval = False, fignum = 0):
    # Compute approximate root with bisec. alg. 
    (a_n, b_n, c) = compute_bisection(f, a, b, n)
    
    #############################################################
    # I cannot emphasize enough that the whole of the scientific
    # computation for approximating the root of f occurs in the 
    # single line of code above that calls the separate function
    # compute_bisection.
    #############################################################
    
    ###############################################################
    # Everything below is just to make fancy illustrative plots of 
    # the results. Read through it carefully. Add/edit comments
    # as you do to make sure you understand what each line of code
    # is doing, but keep in mind that all the scientific computation
    # was already done above. We often put significant effort into
    # visualizing results so that we can properly "tell the story"
    # of the solution.
    ################################################################
    fig = plt.figure(figsize=(10,5), num=fignum)
    x = np.linspace(x_min, x_max, grid_pts)
    
    # First, plot function and axes
    plt.plot(x, f(x), 'b', linewidth=2)  # plot of function f
    plt.axvline(0, linewidth=1, linestyle=':', c='k')  # plot typical y-axis
    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    # Title information
    title_str = 'Approx. root is ' + '{:.6}'.format(c) + '. Use this to update interval [a,b].'
    plt.title(title_str, fontsize=14)
    
    # Create a dict (dictionary) of properties of items used in annotations
    bbox_pt = dict(boxstyle='round', fc='0.9')
    bbox_text = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2) 
    arrowprops = dict(arrowstyle='->', facecolor='black', alpha=0.5, linewidth=2) 
    y_min, y_max = plt.ylim()  # get min and max y-values from plot to offset annotations
    y_range = y_max - y_min
    x_range = x_max - x_min
    
    # Now plot relevant points and annotate
    
    # First annotate parts of the plot involving the points a and b
    if show_final_interval:
        plt.scatter(a_n,f(a_n),s=70,c='k')
        
        plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(a_n,f(a_n)), xytext=(a_n+0.2*x_range,f(a_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b_n,f(b_n),s=70,c='k')
        
        plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(b_n,f(b_n)), xytext=(b_n+0.2*x_range,f(b_n)-0.35*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
        
    else:
        plt.scatter(a,f(a),s=70,c='k')
        
        plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(a,f(a)), xytext=(a+0.1*x_range,f(a)-0.25*y_range),
                     bbox=bbox_pt, arrowprops=arrowprops)
        
        plt.scatter(b,f(b),s=70,c='k')
        
        plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(b,f(b)), xytext=(b+0.1*x_range,f(b)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)

    # Now annotate the part of the plot involving the approximate root c    
    plt.scatter(c,f(c),s=70,c='r')
    
    plt.annotate('Approx. root\nfrom ' + str(n) + '\nstep(s) of bisec.\nalgorithm', fontsize=12, xy=(c,f(c)), 
                     xytext=(c-0.2*x_range,f(c)+0.25*y_range), color='w',
                     bbox = bbox_text, arrowprops=arrowprops)
    
    plt.annotate(r'$(c,f(c))$', fontsize=12, xy=(c,f(c)), xytext=(c+0.1*x_range,f(c)-0.25*y_range),
                 bbox=bbox_pt, arrowprops=arrowprops)
    plt.show()
    return

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

interact_manual(plot_bisection, 
         f = widgets.fixed(lambda x: x**3-x-2),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         grid_pts = widgets.fixed(100),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(0))

In [ ]:
%reset -f out

%matplotlib widget

interact_manual(plot_bisection, 
         f = widgets.fixed(lambda x: np.sin(x**3-x-2)),
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(0),
         x_max = widgets.fixed(3),
         grid_pts = widgets.fixed(1000),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(1))

### Expanding on the `Bisection` class (and **not** the `PlotToolsForRoots` class)
---

<mark> Run the code cell below and click the "play" button to see a recorded lecture associated with this part of the notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('TBD', width=800, height=450)

Let's expand the `Bisection` class to include this useful plotting function that visualizes the steps in the bisection algorithm.

Why do we want to put it in the `Bisection` class and not in the `PlotToolsForRoots` class? 

The plotting function we are interested in coding fundamentally relies upon the outputs of the bisection algorithm (it is the first step in the function). Since we may use `PlotToolsForRoots` to help us setup other root-finding algorithms other than the bisection algorithm (in fact, we do just this below), we do not want to clutter this class with specific plotting functions that will not work when used inside of other classes defined by other root-finding methods. 

<mark> Suggested activity:</mark> 

- Add docstrings and code comments below.

In [ ]:
class Bisection(object):
    
    def __init__(self, f, a=None, b=None, n=None, tol_interval=1e-5, tol_f=1e-8):
        
        self.f = f
        self.a = a
        self.b = b
        self.n = n
        self.tol_interval = tol_interval
        self.tol_f = tol_f
        self.plot_tool = PlotToolsForRoots(f)
        
        return
        
    def set_bisection_parameters(self, a=None, b=None, n=None, tol_interval=None, tol_f=None):
        
        if a is not None:
            self.a = a
        if b is not None:
            self.b = b
        if n is not None:
            self.n = n
        if tol_interval is not None:
            self.tol_interval = tol_interval
        if tol_f is not None:
            self.tol_f = tol_f
            
        return
            
    def compute_bisection(self):
        '''
        This simple function applies up to n iterations of the 
        bisection algorithm. 
        
        It sets new data attributes based on the outputs of the previously 
        defined compute_bisection standalone function.
        '''
        
        if (self.a is None) or (self.b is None) or (self.n is None):
            raise AttributeError('Check that all bisection parameters are set. \n' +\
                   'Use set_bisection_parameters to set: a, b, and/or n.')
            return

        # Let us first check the conditions to apply the bisection
        # algorithm are satisfied
        if self.a >= self.b:
            raise ValueError('The bisec. alg. requires a<b')
        if self.f(a)*self.f(b) > 0:
            raise ValueError('The bisec. alg. requires f(a) and f(b)' +\
                             'are different signs')

        a_current = self.a
        b_current = self.b
        # Now, perform the bisection algorithm
        current_iter = 0
        while current_iter < self.n:
            current_iter += 1

            # bisect the interval [a,b], i.e., compute mid-point
            c = (b_current+a_current)/2 

            # Check if c is an (approx.) root
            if np.abs(self.f(c)) < self.tol_f:
                break

            # Determine if a or b should be replaced with c
            if self.f(a)*self.f(c) < 0:
                b_current = c
            else:
                a_current = c

            # Check if dividing [a,b] in half made the interval 
            # too small to continue
            if (b_current-a_current) < self.tol_interval:
                break
                
        # New data attributes are set based on the completion of the above algorithm        
        self.a_n = a_current  
        self.b_n = b_current
        self.root = c
        self.plot_tool.set_root(c)  # Useful for plotting!

        return
    
    def plot_bisection(self, x_min, x_max, a = None, b = None, n = None, grid_pts = 100, 
                       show_final_interval = False, fignum = 0):
        '''
        Plot information related to the performance of the bisection algorithm
        '''
        
        # Update bisection with any new a, b, or n values
        self.set_bisection_parameters(a=a, b=b, n=n)
        
        # Compute approximate root with bisec. alg. 
        self.compute_bisection()

        self.plot_tool.plot(x_min=x_min, x_max=x_max, N=grid_pts, fignum=fignum, show_root=False)
    
        self.plot_bisection_annotate(x_min, x_max, show_final_interval)
        
        return

    def plot_bisection_annotate(self, x_min, x_max, show_final_interval):
        '''
        Annotate the information plotted related to the performance of the bisection algorithm
        '''
        
        # Create a dict (dictionary) of properties of items used in annotations
        bbox_pt = dict(boxstyle='round', fc='0.9')
        bbox_text = dict(boxstyle="round", fc='b', color='r', alpha=0.25, linewidth=2) 
        arrowprops = dict(arrowstyle='->', facecolor='black', alpha=0.5, linewidth=2) 
        y_min, y_max = plt.ylim()  # get min and max y-values from plot to offset annotations
        y_range = y_max - y_min
        x_range = x_max - x_min

        # Now plot relevant points and annotate

        # First annotate parts of the plot involving the points a and b
        if show_final_interval:
            plt.scatter(self.a_n,self.f(self.a_n),s=70,c='k')
            
            plt.scatter(self.b_n,self.f(self.b_n),s=70,c='k')

            if self.f(self.a) > 0:
                plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(self.a_n, self.f(self.a_n)), 
                         xytext=(self.a_n-0.25*x_range, self.f(self.a_n)-0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(self.b_n, self.f(self.b_n)), 
                         xytext=(self.b_n+0.25*x_range, self.f(self.b_n)+0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
            else:
                plt.annotate(r'$(a_n,f(a_n))$', fontsize=12, xy=(self.a_n,self.f(self.a_n)), 
                         xytext=(self.a_n-0.25*x_range, self.f(self.a_n)+0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b_n,f(b_n))$', fontsize=12, xy=(self.b_n,self.f(self.b_n)), 
                         xytext=(self.b_n-0.25*x_range, self.f(self.b_n)-0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)

        else:
            plt.scatter(self.a,self.f(self.a),s=70,c='k')
            
            plt.scatter(self.b,self.f(self.b),s=70,c='k')

            if self.f(self.a) > 0:
                plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(self.a,self.f(self.a)), 
                         xytext=(self.a+0.1*x_range, self.f(self.a)-0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(self.b,self.f(self.b)), 
                         xytext=(self.b-0.25*x_range, self.f(self.b)+0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
            else:
                plt.annotate(r'$(a,f(a))$', fontsize=12, xy=(self.a,self.f(self.a)), 
                         xytext=(self.a+0.1*x_range, self.f(self.a)+0.1*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)
                
                plt.annotate(r'$(b,f(b))$', fontsize=12, xy=(self.b,self.f(self.b)), 
                         xytext=(self.b-0.25*x_range, self.f(self.b)-0.05*y_range),
                         bbox=bbox_pt, arrowprops=arrowprops)

        # Now annotate the part of the plot involving the approximate root   
        plt.scatter(self.root,self.f(self.root),s=70,c='r')

        plt.annotate('Approx. root\nfrom ' + str(self.n) + '\nstep(s) of bisec.\nalgorithm', fontsize=12, 
                     xy=(self.root,self.f(self.root)), 
                     xytext=(self.root-0.25*x_range, self.f(self.root)+0.2*y_range), color='w',
                     bbox = bbox_text, arrowprops=arrowprops)
        
        plt.annotate(r'$(c,f(c))$', fontsize=12, xy=(self.root,self.f(self.root)),
                     xytext=(self.root+0.1*x_range,self.f(self.root)-0.1*y_range),
                     bbox=bbox_pt, arrowprops=arrowprops)
        
        return

In [ ]:
f_new = lambda x: x**3-x-2

f_bisect = Bisection(f_new)

f_bisect.set_bisection_parameters(a = -1, b = 3, n = 10)

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

f_bisect.plot_bisection(-1, 3)

In [ ]:
# This magic command below with help "flush" outputs to keep memory sizes small if this cell is repeatedly executed
%reset -f out

# Updated to interact_manual to avoid flickering of plots.
# Click on "Run Interact" for each new choice of parameters.
%matplotlib widget

%reset -f out

interact_manual(f_bisect.plot_bisection, 
         a = widgets.FloatSlider(value=0.5, min=-1, max=1.52, step=0.01),
         b = widgets.FloatSlider(value=1.75, min=1.53, max=2, step=0.01),
         n = widgets.IntSlider(value=1, min=1, max=20),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(3),
         grid_pts = widgets.fixed(1000),
         show_final_interval = widgets.Checkbox(False),
         fignum = widgets.fixed(0))

---

## <mark>Activity 1: Playing with the bisection algorithm</mark><a name='activity-bisection'></a>
    
**Note 1**: You may want/need to create additional code (and Markdown) cells below as you work your way through this activity.
    
**Note 2**: You will probably need to explore some plots of the functions first (e.g., by experimenting with the `plot_tool` attribute of instantiations of the `Bisection` class)  to determine viable candidates for the values of `a` and `b` in order to capture the root for the functions you are playing with. Suppose the root is $100$, then you need to setup the problem and any associated plots around this value to get things to work properly. 
    
**Note 3**: It is okay to show examples that produce errors. This is in fact *encouraged* as long as you explain the errors. But, you should also show results that do not produce errors.

**In this activity, you should do the following (by the way, "some" always means at least two):**    
    
- Create instantiations of the `Bisection` class for **some** (meaning at least two) `lambda` functions of your choosing that have roots.
<br><br>
  - Demonstrate approximations of these roots for each `lambda` function using the `plot_bisection` method attribute of the associated instantiation of the `Bisection` class. At least one of these plots should be without widgets and at least one of these plots should use widgets for interactivity.
<br><br>    
  - Summarize results in a Markdown cell you create below the plots.

Notice that the method attribute `plot_bisection` does not have `tol_interval` or `tol_f` as parameters. When it calls `compute_bisection`, it is simply relying upon default values for these parameters. We should improve upon that.


- **Copy/paste the `Bisection` class below and rename it `BisectionImproved`.** 
<br><br>
  - Update the `plot_bisection` method attribute that allows for all of the parameters in the `set_bisection_parameters` method to be set by the user.
  
*Hint:* This requires updating just two things. First, the parameters passed to the method `plot_bisection`. Second, the call to `set_bisection_parameters` within this method.



- Redo the first part of this activity utilizing your `BisectionImproved` class for **some** `lambda` functions of your choosing both **with and without widgets** where you show results for different choices of these new tolerance parameters. 
    
*Hint:* **You should use a FloatLogSlider for the tolerance parameters** since we often vary tolerance parameters by orders of magnitude when we are testing out an algorithm.

End of Activity 1.

---

---
## <mark>Activity 2: Summary for Root-finding</mark> <a name='activity-summary-roots'></a>

Summarize some of the key takeaways/points from this notebook *so far* in regards to root-finding in a list below and prepare a few code examples related to these takeaways/points in the code cells below. You need to have at least one example for each of your summary points and you need at least three summary points.

- [Your summary point 1 goes here]


- [Your summary point 2 goes here]




- [Your summary point 3 goes here]

End of Activity 2.

---

## Part (a)(ii): Numerical integration <a name='integration'></a>

<mark> Run the code cell below and click the "play" button to see a recorded lecture associated with this part of the notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('TBD', width=800, height=450)

### What is integration and why should we care?
---

What *the integral* of a function means depends on context, which also helps to explain the *why* of an integral. 
Typically, it refers to transforming a function into a scalar quantity that describes some type of important aggregate behavior of the function over a set.
It is *kind of like* summing up the behavior of a function over a set in order to make important inferences. 
Some examples of what integrals mean in different contexts are given below. 

- In probability theory, the functions that quantify scalar outputs of an experiment are called random variables. 
The integrals of random variables weighted by their probability density functions give the expected value of the experiment.
Other standard statistical quantities such as variance also involve integrals.

- In engineering design and manufacturing processes, it is often important to compute the length of a curve, the area of a region, or the volume of an object (e.g. to determine the amount of resources/cost in constructing an object). Such quantities are given by integrals.

- In physics, integrals are used to determine important quantities like velocity (which is given as an integral of acceleration) and displacement (an integral of velocity). 

- In finance, integrals are sometimes used to determine options pricing.

- Many models of complex physical phenomena involve partial differential equations that are solved via numerical methods (e.g., finite element methods) that require computations of many integrals to construct accurate approximations. 

- In data science, statistical/machine learning, and any other data-driven discipline where proposed models are *fitted* to data, the *goodness of fit* of a model, a loss function, and other important quantities can usually be described in terms of an integral (or its discrete counterpart: summation). 

<mark> ***Notation:*** </mark>

There are a few different notational conventions, but here we will use the following: Let $f(x)$ be a function and $A\subset dom(f)$ (i.e., $A$ is some set of acceptable inputs taken from the domain of the function $f$), then the integral of $f$ over $A$ is denoted by

$$
  \large  \int_A f(x)
$$

### A standard motivating example
---

In order to build intuition while avoiding complicating calculus details, we consider some simple examples that also serve to make all of this less abstract and more concrete. 

<mark> ***General details:*** </mark>

- Suppose $v(t)$ describes the velocity of an object moving either forward/backward on some path over the time interval $[t_0,t_f]$ (here $t_0$ denotes an initial time and $t_f$ denotes the final time).

- Suppose we are interested in how far along the path the object ultimately ends up relative to its starting position, which is given by

$$
  \large  \int_{[t_0,t_f]} v(t)
$$

<mark> ***Examples with intuitive solutions:*** </mark>

For simple functions of $v(t)$, we can easily conceptualize the problem and intuit the solution with little difficulty (i.e., without referring to anything from calculus). Consider the following scenarios:

- $v(t)=0$ (i.e., the object is not moving). 

    - Then, $\int_{[t_0,t_f]} v(t) = 0$, which means the final displacement of the object from its initial position is zero units of length. Not surprising. The object did not move.<br><br>

- Suppose now that <br><br>
$$
    v(t) = \begin{cases}
                5, & t_0<t<\frac{t_f+t_0}{2}, \\
                -5, & \frac{t_f+t_0}{2}<t<t_f,
            \end{cases}
$$ <br>
   which simply means that the object is moving forward at a constant speed of 5 (ignoring units) for half the time and then moving at the same speed *but backwards* (i.e., in the other direction) the other half of time. Then, $\int_{[t_0,t_f]} v(t) = 0$ because the object just did a simple "round trip" back to where it started. 

- Suppose we have the same velocity as the previous example, but we instead we wanted to know the *total distance* traveled instead of just the final displacement from the starting position? Then, we would want to know $$\large \int_{[0,t_f]} \vert v(t) \vert$$ which means we want to integrate the absolute value of velocity. 
   <br><br>
   
   But, what is $\int_{[t_0,t_f]}\vert v(t) \vert$?
   <br><br>
   
   In this case, we know that this means we are integrating a function that is constant over all time (in this case a constant 5). If an object moves at a constant *speed* (speed is the absolute value of velocity), then we should be able to figure out the total distance it traveled. This is easier when considering units. Suppose $v(t)$ is described in miles per hour, which we rewrite as $\frac{\text{miles}}{\text{hour}}$. It sure seems like if we just multiplied by the total number of hours the object was moving, then we would get the total number of miles the object traveled because 
   
   $$
       \frac{\text{miles}}{\text{hour}}\text{hour} = \text{miles},
   $$
   
    i.e., the "hours cancel out."
   <br><br>
   
   So, in this example, $\int_{[t_0,t_f]} \vert v(t) \vert = 5(t_f-t_0)$.
   <br><br>
   
   Let's visualize these last two results.

In [ ]:
def v(t,t_f):
    n = len(t)
    vs = np.zeros(n)
    for i in range(n):
        if t[i]<t_f/2:
            vs[i] = 5
        else:
            vs[i] = -5
    return vs

In [ ]:
t_0 = 0
t_f = 10  # integral limits
t = np.linspace(t_0, t_f)

%matplotlib widget
fig, ax = plt.subplots(num=0)
ax.plot(t, np.abs(v(t,t_f)), 'r', linewidth=2)
ax.set_ylim(bottom=0)

# Make the shaded region associated with the integral
it = np.linspace(t_0, t_f)
iv = np.abs(v(it,t_f))
verts = [(t_0, 0), *zip(it, iv), (t_f, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

ax.text(0.5 * (t_0 + t_f), 2.5, r"$\int_{[t_0,t_f]} \vert v(t)\vert=5(t_f-t_0)$",
        horizontalalignment='center', fontsize=20)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks((t_0, t_f))
ax.set_xticklabels(('$t_0$', '$t_f$'), fontsize=20);

In [ ]:
t_0 = 0
t_f = 10  # integral limits
t = np.linspace(t_0, t_f, 101)

%matplotlib widget
fig, ax = plt.subplots(num=1)
ax.plot(t, v(t,t_f), 'r', linewidth=2)

# Make the shaded region associated with the integral
it = np.linspace(t_0, t_f, 101)
iv = v(it,t_f)
verts = [(t_0, 0), *zip(it, iv), (t_f, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

ax.text(0.75 * (t_0 + t_f), 2.5, r"$\int_{[t_0,t_f]} v(t)=0$",
        horizontalalignment='center', fontsize=20)

ax.axhline(0, linewidth=1, c='k')  # plot $v=0$ line to more clearly demonstrate the positive/negative parts

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks((t_0, t_f))
ax.set_xticklabels(('$t_0$', '$t_f$'), fontsize=20);

### What do we see?
---

- It appears that these integrals are related to the *areas* of the shaded rectangles. 

    In particular, $\int_{[t_0,t_f]} |v(t)|$ is exactly the area of the single shaded rectangle.


- On the other hand, $\int_{[t_0,t_f]} v(t)$ is given by the area of the rectangle above the $t$-axis *minus* the area of the rectangle below the $t$-axis. They appear to *cancel* each other out. In fact, if we think of areas as being *signed* so that any area above a horizontal axis is positive and any area below the horizontal axis is negative, then we see that $\int_{[t_0,t_f]} v(t)$ is actually the sum of the *signed* areas. 


- These observations actually lead us to a useful conceptualization of integrals in terms of "sizes" of positive and negative regions of a function over a set. 

Below, we visualize what the sign of an integral is for a function in terms of what dominates: the positive or negative areas `quad` function to compute accurate approximations of the integral.

<mark> ***Key takeaways about `quad`:*** </mark>

- The `quad` function is within the `integrate` subpackage of `scipy`, and you should take at least 5-10 minutes to review some of the documentation https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html

- The `quad` function returns a tuple as an output (much like our `compute_bisection` function from the previous lecture notebook). 
The first component is the approximation of the integral. The second component is an approximation of the error in the integral. Therefore, we usually append a `[0]` to the end of it whenever we are just interested in the actual approximation of the integral (you should take note of this below).

- If you peruse the source code for the `quad` function, then you will see some nice docstrings, doctests, etc. https://github.com/scipy/scipy/blob/v1.5.3/scipy/integrate/quadpack.py#L49-L442

The `integrate` subpackage is itself rather interesting and useful to familiarize yourself with: https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html

In [ ]:
from scipy.integrate import quad

In [ ]:
def plot_integral(f, x_min, x_max, a, b, fignum=0):
    # Estimate the integral with quad function
    # The quad returns a tuple and the first component is
    # the estimate of the integral we are after
    int_ab = quad(f, a, b)[0] 
    
    #### EVERYTHING ELSE BELOW IS JUST PLOTTING
    x = np.linspace(x_min, x_max, 101)
    y = f(x)

    plt.figure(num=fignum)
    plt.clf()
    fig, ax = plt.subplots(num=fignum)
    ax.plot(x, y, 'r', linewidth=2)

    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    # Make the shaded region
    ix = np.linspace(a, b, 101)
    iy = f(ix)
    # A * before the zip will "unpack" the tuples in the 
    # list of tuples created by zip
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)
    
    ax.set_title(r"$\int_a^b f(x)\mathrm{d}x \approx $ %3.2f" %int_ab, fontsize=10)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    plt.show()

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_integral, 
         f = widgets.fixed(lambda x: (x + 1) * (x - 5) * np.sin(3*x)),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         fignum = widgets.fixed(0));

**Creating an `Integration` class.**

<mark> Suggested activity:</mark> 
- Add docstrings to the `Integration` class below.

- Add code comments to this class.

In [ ]:
class Integration(object):
    '''
    
    '''
    def __init__(self, f, a=None, b=None):
        '''
        
        '''
        self.f = f
        self.a = a
        self.b = b
        
        return
    
    def set_integral_limits(self, a, b):
        '''
        
        '''
        self.a = a
        self.b = b
        
        return
    
    
    def evaluate_integral(self, a=None, b=None):
        '''
        
        '''
        if (a is not None) and (b is not None):
            self.set_integral_limits(a, b)
            
        self.integral = quad(self.f, self.a, self.b)[0]
        
        return self.integral
    
    def plot_integral(self, x_min=None, x_max=None, N=101, fignum=0):
        '''
        
        '''
        if x_min is None:
            x_min = self.a
        
        if x_max is None:
            x_max = self.b
            
        x = np.linspace(x_min, x_max, N)
        y = self.f(x)

        plt.figure(num=fignum)
        plt.clf()
        fig, ax = plt.subplots(num=fignum)
        ax.plot(x, y, 'r', linewidth=2)

        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

        # Make the shaded region
        ix = np.linspace(self.a, self.b, N)
        iy = self.f(ix)
        # A * before the zip will "unpack" the tuples in the 
        # list of tuples created by zip
        verts = [(self.a, 0), *zip(ix, iy), (self.b, 0)]
        poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
        ax.add_patch(poly)

        ax.set_title(r"$\int_a^b f(x)\mathrm{d}x \approx $ %3.2f" %self.integral, fontsize=10)
            
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks((self.a, self.b))
        ax.set_xticklabels(('$a$', '$b$'))
        plt.show()
        
        return
        
    def evaluate_and_plot_integral(self, a=None, b=None, x_min=None, x_max=None, fignum=0):
        '''
        
        '''
        self.evaluate_integral(a=a, b=b)
        
        self.plot_integral(x_min=x_min, x_max=x_max, fignum=fignum)
        
        return

In [ ]:
f_integral = Integration(lambda x: (x + 1) * (x - 5) * np.sin(3*x))

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(f_integral.evaluate_and_plot_integral, 
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         N = widgets.fixed(101), 
         fignum = widgets.fixed(0));

### Context is everything and these are just 1-dimensional examples
---

Above, we have just looked at functions with univariate inputs. Subsequently, we observed that the integral is related to the *signed* area between the curve defined by the function and the axis defined by the input.

This observation can be conceptually extended to functions with multivariate inputs. Suppose $f(x)$ is a real-valued function, but $x=(x_1,x_2)$ is a point in $\mathbb{R}^2$. Then, for a set $A\subset\mathbb{R}^2$, the integral $\int_A f(x)$ is related to the *signed volume* between the *surface* given by the graph of $f(x)$ over $A$ and the horizontal *plane* defined by the 2-dimensional input.

Generalizing the concept of area/volume is necessary to describe integrals in geometric terms when the inputs are three-dimensional or higher. 

We can pretty much stick with 1-dimensional examples to explore the typical numerical algorithms used to estimate integrals although some of the strengths and weaknesses of these algorithms are not apparent until we get to higher dimensions. Keep that in mind.

## Geometric (deterministic) methods <a name='rectangle-rules'></a>

<mark> Run the code cell below and click the "play" button to see a recorded lecture associated with this part of the notebook.</mark>

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('GyWZKfm6y2A', width=800, height=450)

There are lots of algorithms to perform numerical integration, e.g., see https://en.wikipedia.org/wiki/Numerical_integration.

We focus on a geometric method based upon rectangles. In the external activities, we consider a stochastic form of integration called Monte Carlo integration. Monte Carlo methods are at the heart of many data science algorithms that rely upon some sort of stochastic implementation.

Many of the deterministic approaches to estimating integrals are based on partitioning $A$ (i.e., cutting up the set $A$ into a non-overlapping collection of subsets although we allow for shared boundaries between the subsets) such that the signed "areas" (or "volume" or generalizations of volume) between the graph of $f(x)$ and these subsets drawn on the horizontal axis (or plane or hyperplane) defined by the inputs are well-approximated by a simple geometric object for which we know the area (or volume or generalization of volume).

<mark> ***Key Points:*** </mark>

- The simplest such methods for 1-D problems involve the use of rectangles or trapezoids where the areas are easily computed from rules we learned in geometry.


- When $A$ is simply an interval, we typically choose the partition of $A$ to be equally sized sub-intervals and approximate the signed area of $f(x)$ over each subinterval using a rectangle that is as wide as the sub-interval with height given by the evaluation of $f(x)$ at some point in the sub-interval. 


- Adding up all the signed areas of rectangles gives an approximation to the integral. This is visualized below where we provide some options about where we can evaluate the function $f(x)$. 

<mark> Suggested activity:</mark> 

- Add a useful docstring to `compute_rect_rules` to describe the role of the various parameters. 

- Add some doctests.

In [ ]:
def compute_rect_rules(f, a, b, n, rule='left'):
    '''
    Add a useful docstring here and doctest!
    '''
    ix = np.linspace(a, b, n+1)  # create the x-values to create the base of each rectangle 
    Delta_x = ix[1]-ix[0]  # compute the length of the base of each rectangle
    
    # Now determine the height of the rectangle by evaluating the
    # function f at some point along its base.
    if rule == 'left':
        iy = f(ix[0:-1])  # evaluate function at left-hand side of its base
    elif rule == 'right':
        iy = f(ix[1:])  # evaluate the function at right-hand side of its base
    else:
        iy = f(ix[0:-1]+0.5*Delta_x)  # evaluate the function at midpoint of its base
    
    # Now compute the approximate integral by adding up all 
    # the areas of the rectangles
    int_approx = np.sum(iy)*Delta_x
    
    return (ix, Delta_x, iy, int_approx)

***Let's do some numerical experiments and analysis.***

Below, we show how to

1. Use the `compute_rect_rules` function defined above with the default rule.

2. Analyze the rate of convergence (ROC).

3. Create interactive visualizations of the `compute_rect_rules` function that help us tell a story.

In [ ]:
f = lambda x: (x + 1) * (x - 5) * np.sin(3*x) 

In [ ]:
# This shows how to use the lambda function f above as well as the default rule
# to get an approximation
int_approx = compute_rect_rules(f, a=2, b=4, n=10)[3]

# Print the approximate integral
print(int_approx)

To study the ROC for any approximation process, we usually vary critical inputs over orders of magnitude and employ log-log plots to understand how errors are reduced by refinement of the inputs. 

In [ ]:
# The logspace function is an easy way to create an array that varies over various orders of magnitude
# The code below creates ns that go from 10**1 to 10**4
ns = np.logspace(1, 4, num=4) 
print(ns)

In [ ]:
print(ns.dtype)  # Unfortunately, the compute_rect_rules requires the number of rectangles to be of type int

In [ ]:
# An easy fix
ns = np.logspace(1, 4, num=4).astype('int')
print(ns)

In [ ]:
# Let's look at a list of approximations for increasing number of rectangles
int_approxes = []
for n in ns:
    int_approxes.append(compute_rect_rules(f, a=2, b=4, n=n)[3])
print(int_approxes)

In [ ]:
# Now compute the errors as a function of increasing number of rectangles
errors_left_approx = np.array(int_approxes) - quad(f, a=2, b=4)[0]
print(errors_left_approx)

In [ ]:
# Now visualize the convergence of the left-hand rule
plt.figure()
plt.loglog(ns, np.abs(errors_left_approx))
plt.xlabel('# of rects.', fontsize=12)
plt.title('Abs. Value of Errors vs. # of rects (left-hand rule)')

In [ ]:
# How to estimate the rate of convergence (ROC)?

# The above curve is approximately a straight line. We therefore fit a line
# of best fit below and report the slope of that line.

ROC_estimate = np.polyfit(np.log(ns), np.log(np.abs(errors_left_approx)), 1)[0]

# The slope of the line of best fit tells us how many orders of magnitude the error
# is decreased for every order of magnitude increase in the number of rectangles.
print(ROC_estimate) 

***Now we do some fancier visualization.***

In [ ]:
def plot_rectangle_rules(f, a, b, n, rule='left', x_min=0, x_max=1, fignum=0):
    # First get the approximation from a rectangle rule
    ix, Delta_x, iy, int_approx = compute_rect_rules(f, a, b, n, rule)
    
    # Get a more accurate approximation from quad
    # This is not necessary, but we use it for comparison sake
    int_ab = quad(f, a, b)[0] 
    
    ################# EVERYTHING BELOW IS JUST FOR PLOTTING
    x = np.linspace(x_min, x_max, 101)
    y = f(x)

    plt.figure(num=fignum)
    plt.clf()
    fig, ax = plt.subplots(figsize=(10,10), num=fignum)
    ax.plot(x, y, 'r', linewidth=2)

    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    rects = []
    for i in range(n):
        rects.append(Rectangle((ix[i],0), Delta_x, iy[i]))    
    # Create patch collection with specified colour/alpha
    pc = PatchCollection(rects, facecolor='0.9', alpha=0.5,
                         edgecolor='0.5')
    ax.add_collection(pc)
   
    ax.set_title(r"$\int_a^b f(x)\mathrm{d}x = %3.2f \approx $ Sum of Rect. Areas = %3.2f" 
                 %(int_ab, int_approx), fontsize=20)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    plt.show()

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_rectangle_rules, 
         f = widgets.fixed(lambda x: (x + 1) * (x - 5) * np.sin(3*x)),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=3, min=1, max=50),
         rule=['left', 'right', 'middle'],
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         fignum=widgets.fixed(0));

***We visualize approximations of the integral for a different function below.***

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_rectangle_rules, 
         f = widgets.fixed(lambda x: x**2),
         a = widgets.FloatSlider(value=0, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=1, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=1000),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         rule=['left', 'right', 'middle'],
         fignum=widgets.fixed(0));

***Let's make a `RectangleRule` `class` as a subclass of `Integration`.***

In [ ]:
class RectangleRule(Integration):
    '''
    
    '''
    def __init__(self, f, a=None, b=None, n=None, rule='Left'):
        '''
        
        '''
        super().__init__(f=f, a=a, b=b)
        self.n = n
        self.rule = rule
        
        return
        
    def change_rectangles(self, n=None, rule=None):
        '''
        
        '''
        if n is not None:
            self.n = n
        if rule is not None:
            self.rule = rule
            
        return
    
    def compute_integral_approx(self):
        '''
        
        '''
        self.ix = np.linspace(self.a, self.b, self.n+1)  # create the x-values to create the base of each rectangle 
        self.Delta_x = self.ix[1]-self.ix[0]  # compute the length of the base of each rectangle

        # Now determine the height of the rectangle by evaluating the
        # function f at some point along its base.
        if self.rule == 'left':
            self.iy = self.f(self.ix[0:-1])  # evaluate function at left-hand side of its base
        elif self.rule == 'right':
            self.iy = self.f(self.ix[1:])  # evaluate the function at right-hand side of its base
        else:
            self.iy = self.f(self.ix[0:-1]+0.5*self.Delta_x)  # evaluate the function at midpoint of its base

        # Now compute the approximate integral by adding up all 
        # the areas of the rectangles
        self.integral_approx = np.sum(self.iy)*self.Delta_x
        
        return self.integral_approx
    
    
    def plot_rectangle_approx(self, x_min=0, x_max=1, N=101, fignum=0):
        
        # Get a more accurate approximate for comparison sake using method
        # inherited from the super class and plot.
        self.evaluate_integral()

        ################# EVERYTHING BELOW IS JUST FOR PLOTTING
        x = np.linspace(x_min, x_max, N)
        y = self.f(x)

        plt.figure(num=fignum)
        plt.clf()
        fig, ax = plt.subplots(num=fignum)
        ax.plot(x, y, 'r', linewidth=2)
        
        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

        rects = []
        for i in range(self.n):
            rects.append(Rectangle((self.ix[i],0), self.Delta_x, self.iy[i]))    
        # Create patch collection with specified colour/alpha
        pc = PatchCollection(rects, facecolor='0.9', alpha=0.5,
                             edgecolor='0.5')
        ax.add_collection(pc)

        ax.set_title(r"$\int_a^b f(x)\mathrm{d}x = %3.2f \approx $ Sum of Rect. Areas = %3.2f" 
                     %(self.integral, self.integral_approx), fontsize=20)

        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks((self.a, self.b))
        ax.set_xticklabels(('$a$', '$b$'))
        plt.show()
        
        return
    
    def evaluate_and_plot_rectangle_approx(self, a, b, n, rule, x_min, x_max, N=101, fignum=0):
        '''
        
        '''
        self.change_rectangles(n=n, rule=rule)
        
        self.set_integral_limits(a=a, b=b)
        
        self.compute_integral_approx()
        
        self.plot_rectangle_approx(x_min=x_min, x_max=x_max, N=N, fignum=fignum)
        
        return

In [ ]:
f_rects = RectangleRule(f = lambda x: (x + 1) * (x - 5) * np.sin(3*x))

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(f_rects.evaluate_and_plot_rectangle_approx, 
         a = widgets.FloatSlider(value=0, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=1, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=1000),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         rule=['left', 'right', 'middle'],
         N = widgets.fixed(101),
         fignum=widgets.fixed(0));

---

## <mark>Activity 3: Rectangle rules</mark><a id='activity-rectangle-rules'></a>

*For the sake of simplicity, this activity does not require the use of any classes.*

Feel free to create many new code and markdown cells below as you work through this activity.

- Create a wrapper function, `multiple_rect_approx`, which has all the same parameters as `compute_rect_rules` except that the `n` parameter is now assumed to be an *array* (or *list*) of integers containing different numbers of rectangles for which the approximate integrals should be computed. 

  This wrapper function should loop through all the values in the list of `n` values, call `compute_rect_rules` to obtain the associated approximate integral, and store all of the approximate integrals in a list that is returned by the wrapper function.

- For a number of different `lambda` functions including `f = lambda x: x`, `f = lambda x: x**2`, and at least one of your own choosing, compute the errors for the left-, right, and midpoint-rule approximations for `n=[10, 100, 1000]` for `a=0` and `b=1`. Store the errors as either a list or numpy array. 

    *Hint*: To compute the errors, you need to either know the exact value (or a very accurate approximation) of the integral of the function to compare to the rectangle rule approximations. Use the imported `quad` function from `scipy.integrate` as the "exact" value (it is a very good approximation in almost all cases).

- For each of your different `lambda` functions, create a log-log plot with a legend that shows the errors for each rectangle rule approximation vs. the number of rectangles. A log-log plot of errors is useful for analyzing the rate of convergence of a method. We like to see the errors *decrease* as we *increase* the computational effort. Estimate the rate of convergence using the `np.polyfit` function as shown in the example above.

   *Hint*: To use a log-log plot, everything must be non-negative, so you should use the `np.abs` function on the y-values of this plot as shown above.

- Comment on/interpret your results in the Markdown cell below. Some questions you should try to answer are the following. Which method appears to be most accurate/converge faster? Why do you think that is? *Hint:* Creating visualizations of the approximations of integrals with a small number of rectangles for the different methods may provide insight into how certain methods may be more likely to have "canceling" or "off-setting" errors within each rectangle. Some brief research online will also reveal the answer.

End of Activity 3.

---

---
## <mark>Activity 4: Summary for Integration</mark> <a name='activity-summary-integration'></a>

Summarize some of the key takeaways/points from this notebook in regards to integration in a list below and prepare a few code examples related to these takeaways/points in the code cells below. You need to have at least one example for each of your summary points and you need at least three summary points.

- [Your summary point 1 goes here]


- [Your summary point 2 goes here]




- [Your summary point 3 goes here]

End of Activity 4.

---

## [Click here to return to Notebook Contents](#Contents)